<a href="https://colab.research.google.com/github/FarrelAD/Tim-Maju-Lancar-Datathon-Jatim-2025/blob/main/KerangkaAnalisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd # Ensure pandas is imported
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix


uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving jatim-excel.xlsx to jatim-excel.xlsx
User uploaded file "jatim-excel.xlsx" with length 157583 bytes


In [ ]:
df = pd.read_excel(list(uploaded.keys())[0])
display(df.head(100))

,kabupaten/kota,kecamatan,sekolah,rombel,sekolah_id,npsn,bentuk_pendidikan,status_sekolah,akreditasi,guru_kelas,...,before_tempat_bermain_olahraga,after_tempat_bermain_olahraga,before_ruang_tu,after_ruang_tu,before_ruang_konseling,after_ruang_konseling,before_ruang_osis,after_ruang_osis,before_bangunan,after_bangunan
0,Kab. Bangkalan,Kec. Bangkalan,SLB SAMUDRA LAVENDER,13,58B659F8-2673-4B66-ADC9-0995AFA0E80E,69830686,SLB,Swasta,B,13,...,0,0,0,0,0,0,0,0,2,2
1,Kab. Bangkalan,Kec. Kamal,SLB PGRI,11,4028497E-8D18-E111-8F7E-F5091768A769,20531143,SLB,Swasta,B,6,...,0,0,0,0,0,0,0,0,3,3
2,Kab. Bangkalan,Kec. Socah,SLB NEGERI KELEYAN BANGKALAN,33,60CF4B7E-8D18-E111-A61E-3BDB8AFFE3B0,20531387,SLB,Negeri,B,21,...,0,0,1,1,0,0,0,0,6,6
3,Kab. Bangkalan,Kec. Tragah,SLB SAMUDRA TERRA ATHENA,6,C28A7F37-6226-475E-B414-42822EFD4E03,70027553,SLB,Swasta,B,6,...,1,1,0,0,0,0,0,0,5,5
4,Kab. Banyuwangi,Kec. Bangorejo,SLB PGRI BANGOREJO,13,F709E309-7266-4D2D-9BB0-F11BAB758EDE,69861080,SLB,Swasta,C,9,...,0,0,0,0,1,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Kab. Jombang,Kec. Gudo,SLB SUNAR AJI,12,9B749798-150E-4C7C-8B7F-51AB139ED6AC,69984103,SLB,Swasta,C,3,...,0,0,0,0,0,0,0,0,8,8
96,Kab. Jombang,Kec. Jogoroto,SLB DARUL ULUM,12,6076DCF5-8B18-E111-8B56-1F9EFE82BDE6,20503606,SLB,Swasta,A,7,...,1,1,0,0,0,0,0,0,1,1
97,Kab. Jombang,Kec. Jombang,SLB MUHAMMADIYAH,26,NaN,20503224,SLB,Negeri,NaN,10,...,0,0,2,2,1,1,0,0,17,17
98,Kab. Jombang,Kec. Jombang,SLB NEGERI JOMBANG,24,F034FBF2-8B18-E111-8FFD-8B630529F9BF,20503224,SLB,Negeri,B,12,...,0,0,1,1,0,0,0,0,8,8


#Compare Quantity Swasta vs Negeri

In [ ]:
print("\n=== Jumlah dan Proporsi Sekolah Negeri vs Swasta ===")
status_counts = df['status_sekolah'].value_counts()
status_proportions = df['status_sekolah'].value_counts(normalize=True) * 100

status_summary = pd.DataFrame({
    'Count': status_counts,
    'Proportion (%)': status_proportions.map('{:.2f}%'.format)
})
display(status_summary)


=== Jumlah dan Proporsi Sekolah Negeri vs Swasta ===


,Count,Proportion (%)
status_sekolah,,
Swasta,297,75.77%
Negeri,95,24.23%


#Data Understanding & Preprocessing

In [ ]:
# Filter data for Negeri status only
df_negeri = df[df['status_sekolah'] == 'Negeri'].copy()

# 1. Cek missing values for Negeri schools
missing_values_negeri = df_negeri.isnull().sum()
print("=== Missing Values Per Column (Negeri Schools) ===")
print(missing_values_negeri.to_string())
print("\n=== Missing Values di Kolom 'akreditasi' (Negeri Schools) ===")
print(missing_values_negeri['akreditasi'])

=== Missing Values Per Column (Negeri Schools) ===
kabupaten/kota                    0
kecamatan                         0
sekolah                           0
rombel                            0
sekolah_id                        4
npsn                              0
bentuk_pendidikan                 0
status_sekolah                    0
akreditasi                        4
guru_kelas                        0
guru_matematika                   0
guru_bahasa_indonesia             0
guru_bahasa_inggris               0
guru_sejarah_indonesia            0
guru_pkn                          0
guru_penjaskes                    0
guru_agama_budi_pekerti           0
guru_seni_budaya                  0
ptk_laki                          0
ptk_perempuan                     0
pegawai_laki                      0
pegawai_perempuan                 0
pd_kelas_1_laki                   0
pd_kelas_1_perempuan              0
pd_kelas_2_laki                   0
pd_kelas_2_perempuan              0
pd_kelas_3_la

In [ ]:
# 2. Normalisasi teks kabupaten/kota & kecamatan
df['kabupaten/kota'] = df['kabupaten/kota'].str.lower().str.strip()

In [ ]:
# 3. Rasio guru per siswa

# Calculate teacher-student ratio
df_negeri['teacher_student_ratio'] = df_negeri['ptk'] / df_negeri['pd']

# Handle potential NaN or infinite values in 'teacher_student_ratio'
# Replace infinite values with NaN
df_negeri['teacher_student_ratio'] = df_negeri['teacher_student_ratio'].replace([np.inf, -np.inf], np.nan)
# Fill NaN values with the mean of the column (or another appropriate strategy)
df_negeri['teacher_student_ratio'].fillna(df_negeri['teacher_student_ratio'].mean(), inplace=True)


print("\n=== Rasio Guru per Siswa per Sekolah (Sorted - Negeri Schools) ===")
# Group by sekolah and select the calculated ratio, then sort
teacher_student_ratio_sekolah_sorted = df_negeri.groupby('sekolah')[['ptk', 'pd', 'teacher_student_ratio']].first().reset_index() # Using first() as no aggregation is requested
teacher_student_ratio_sekolah_sorted = teacher_student_ratio_sekolah_sorted.sort_values(by='teacher_student_ratio', ascending=False)
display(teacher_student_ratio_sekolah_sorted)


=== Rasio Guru per Siswa per Sekolah (Sorted - Negeri Schools) ===


/tmp/ipython-input-3380250764.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_negeri['teacher_student_ratio'].fillna(df_negeri['teacher_student_ratio'].mean(), inplace=True)


,sekolah,ptk,pd,teacher_student_ratio
0,SDLB ABCD PGRI 3 CLURING,4,1,4.000000
92,SMALBS BHAKTI PERTIWI,5,13,0.384615
54,SLB NEGERI KELEYAN BANGKALAN,19,50,0.380000
8,SLB AISYIYAH PORONG SIDOARJO,9,24,0.375000
62,SLB NEGERI SAMBIREJO NGANJUK,21,56,0.375000
...,...,...,...,...
12,SLB BERANDA ISTIMEWA GRUDO,6,63,0.095238
25,SLB NEGERI,11,121,0.090909
65,SLB NEGERI SENGON AGUNG PURWOSARI KABUPATEN PA...,7,78,0.089744
3,SLB PGRI 3 CLURING,5,88,0.056818


In [ ]:
# Filter data for Negeri status only
df_negeri = df[df['status_sekolah'] == 'Negeri'].copy()

# 4. Indeks infrastruktur (jumlah ruang, lab, perpustakaan, bangunan)
infrastructure_cols = [
    col for col in df_negeri.columns
    if 'ruang' in col or 'laboratorium' in col or 'perpustakaan' in col or 'bangunan' in col
]
df_negeri['infrastructure_index'] = df_negeri[infrastructure_cols].sum(axis=1)
print("\n=== Indeks Infrastruktur (sample - Negeri Schools) ===")
display(df_negeri[['sekolah'] + infrastructure_cols + ['infrastructure_index']].head())


=== Indeks Infrastruktur (sample - Negeri Schools) ===


,sekolah,before_ruang_kelas,after_ruang_kelas,before_ruang_perpus,after_ruang_perpus,before_ruang_lab,after_ruang_lab,before_ruang_praktik,after_ruang_praktik,before_ruang_pimpinan,...,after_ruang_sirkulasi,before_ruang_tu,after_ruang_tu,before_ruang_konseling,after_ruang_konseling,before_ruang_osis,after_ruang_osis,before_bangunan,after_bangunan,infrastructure_index
2,SLB NEGERI KELEYAN BANGKALAN,6,6,1,1,0,0,0,0,1,...,0,1,1,0,0,0,0,6,6,34
11,SDLB ABCD PGRI 3 CLURING,6,6,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,1,26
12,SLB PGRI 3 CLURING,7,7,1,1,2,2,1,1,1,...,0,1,1,1,1,1,1,1,1,38
13,SLB NEGERI TAMANAGUNG BANYUWANGI,12,12,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,18,18,70
14,SMALBS BHAKTI PERTIWI,8,8,1,1,0,0,0,0,1,...,0,1,1,0,0,0,0,1,1,26


In [ ]:
# Filter data for Negeri status only
df_negeri = df[df['status_sekolah'] == 'Negeri'].copy()

# 5. Rasio gender siswa & guru
df_negeri['student_gender_ratio'] = df_negeri['pd_perempuan'] / df_negeri['pd_laki']
df_negeri['teacher_gender_ratio'] = df_negeri['ptk_perempuan'] / df_negeri['ptk_laki']
print("\n=== Rasio Gender Siswa & Guru (sample - Negeri Schools) ===")
display(df_negeri[['sekolah', 'pd_perempuan', 'pd_laki', 'student_gender_ratio',
          'ptk_perempuan', 'ptk_laki', 'teacher_gender_ratio']].head())


=== Rasio Gender Siswa & Guru (sample - Negeri Schools) ===


,sekolah,pd_perempuan,pd_laki,student_gender_ratio,ptk_perempuan,ptk_laki,teacher_gender_ratio
2,SLB NEGERI KELEYAN BANGKALAN,18,32,0.562500,13,6,2.166667
11,SDLB ABCD PGRI 3 CLURING,1,0,inf,4,0,inf
12,SLB PGRI 3 CLURING,39,49,0.795918,3,2,1.500000
13,SLB NEGERI TAMANAGUNG BANYUWANGI,44,59,0.745763,21,10,2.100000
14,SMALBS BHAKTI PERTIWI,7,6,1.166667,5,0,inf


# Exploratory Data Analysis (EDA)

In [ ]:
# Filter data for Negeri status only
df_negeri = df[df['status_sekolah'] == 'Negeri'].copy()

# 1. Distribusi SLB per kabupaten/kota (This requires geospatial data which is not available, so we'll show a count per district/city)
print("\n=== Jumlah SLB per Kabupaten/Kota (Negeri Schools) ===")
slb_per_kabupaten = df_negeri['kabupaten/kota'].value_counts().reset_index()
slb_per_kabupaten.columns = ['kabupaten/kota', 'jumlah_slb']
display(slb_per_kabupaten)


=== Jumlah SLB per Kabupaten/Kota (Negeri Schools) ===


,kabupaten/kota,jumlah_slb
0,kab. bojonegoro,8
1,kota blitar,6
2,kab. bondowoso,5
3,kab. banyuwangi,5
4,kab. sidoarjo,4
5,kab. jombang,4
6,kab. ngawi,4
7,kab. tuban,4
8,kab. probolinggo,4
9,kab. madiun,4


In [ ]:
# Filter data for Negeri status only
df_negeri = df[df['status_sekolah'] == 'Negeri'].copy()

# 3. Jumlah siswa per jenjang kelas
print("\n=== Jumlah Siswa per Jenjang Kelas (Negeri Schools) ===")
siswa_per_jenjang = df_negeri[[f'pd_kelas_{i}_laki' for i in range(1, 14)] + [f'pd_kelas_{i}_perempuan' for i in range(1, 14)]].sum()
siswa_per_jenjang = siswa_per_jenjang.reset_index()
siswa_per_jenjang.columns = ['kelas_gender', 'jumlah_siswa']
siswa_per_jenjang['jenjang'] = siswa_per_jenjang['kelas_gender'].apply(lambda x: int(x.split('_')[2]))
siswa_per_jenjang['gender'] = siswa_per_jenjang['kelas_gender'].apply(lambda x: x.split('_')[3])
siswa_per_jenjang_total = siswa_per_jenjang.groupby('jenjang')['jumlah_siswa'].sum().reset_index()
display(siswa_per_jenjang_total)


=== Jumlah Siswa per Jenjang Kelas (Negeri Schools) ===


,jenjang,jumlah_siswa
0,1,593
1,2,679
2,3,630
3,4,665
4,5,573
5,6,549
6,7,779
7,8,752
8,9,707
9,10,701


In [ ]:
# Filter data for Negeri status only
df_negeri = df[df['status_sekolah'] == 'Negeri'].copy()

# Calculate ratios *before* handling inf values for display
df_negeri['student_gender_ratio'] = df_negeri['pd_perempuan'] / df_negeri['pd_laki']
df_negeri['teacher_gender_ratio'] = df_negeri['ptk_perempuan'] / df_negeri['ptk_laki']


# 4. Perbandingan gender siswa/guru
print("\n=== Perbandingan Gender Siswa dan Guru (Negeri Schools) ===")

# Handle infinite values in gender ratios
df_negeri['teacher_gender_ratio_handled'] = df_negeri['teacher_gender_ratio'].replace([float('inf'), float('-inf')], float('nan'))
df_negeri['student_gender_ratio_handled'] = df_negeri['student_gender_ratio'].replace([float('inf'), float('-inf')], float('nan'))


# Calculate and display gender ratios in 1:X format for each school
def format_gender_ratio(ratio):
    if pd.isna(ratio):
        return "N/A"
    elif ratio >= 1:
        return f"1:{ratio:.2f}"
    elif ratio > 0 and ratio < 1:
        return f"1:{1/ratio:.2f}"
    elif ratio == 0:
        return "1:Inf" # Or handle as appropriate for zero in numerator
    else:
         return "N/A"

# Apply the formatting function to the handled ratio columns
df_negeri['Student_Gender_Ratio_Formatted'] = df_negeri['student_gender_ratio_handled'].apply(format_gender_ratio)
df_negeri['Teacher_Gender_Ratio_Formatted'] = df_negeri['teacher_gender_ratio_handled'].apply(format_gender_ratio)


# Display the original columns and the new formatted ratio columns
display(df_negeri[['sekolah', 'pd_perempuan', 'pd_laki', 'Student_Gender_Ratio_Formatted',
          'ptk_perempuan', 'ptk_laki', 'Teacher_Gender_Ratio_Formatted']].head())

print(f"\nTotal Siswa Laki-laki (Negeri Schools): {df_negeri['pd_laki'].sum()}")
print(f"Total Siswa Perempuan (Negeri Schools): {df_negeri['pd_perempuan'].sum()}")
print(f"Total Guru Laki-laki (Negeri Schools): {df_negeri['ptk_laki'].sum()}")
print(f"Total Guru Perempuan (Negeri Schools): {df_negeri['ptk_perempuan'].sum()}")


=== Perbandingan Gender Siswa dan Guru (Negeri Schools) ===


,sekolah,pd_perempuan,pd_laki,Student_Gender_Ratio_Formatted,ptk_perempuan,ptk_laki,Teacher_Gender_Ratio_Formatted
2,SLB NEGERI KELEYAN BANGKALAN,18,32,1:1.78,13,6,1:2.17
11,SDLB ABCD PGRI 3 CLURING,1,0,N/A,4,0,N/A
12,SLB PGRI 3 CLURING,39,49,1:1.26,3,2,1:1.50
13,SLB NEGERI TAMANAGUNG BANYUWANGI,44,59,1:1.34,21,10,1:2.10
14,SMALBS BHAKTI PERTIWI,7,6,1:1.17,5,0,N/A



Total Siswa Laki-laki (Negeri Schools): 4979
Total Siswa Perempuan (Negeri Schools): 3118
Total Guru Laki-laki (Negeri Schools): 367
Total Guru Perempuan (Negeri Schools): 1068


In [ ]:
# Filter data for Negeri status only
df_negeri = df[df['status_sekolah'] == 'Negeri'].copy()

# Recalculate necessary columns after filtering
df_negeri['teacher_student_ratio'] = df_negeri['ptk'] / df_negeri['pd']
df_negeri['student_gender_ratio'] = df_negeri['pd_perempuan'] / df_negeri['pd_laki']
df_negeri['teacher_gender_ratio'] = df_negeri['ptk_perempuan'] / df_negeri['ptk_laki']
infrastructure_cols = [
    col for col in df_negeri.columns
    if 'ruang' in col or 'laboratorium' in col or 'perpustakaan' in col or 'bangunan' in col
]
df_negeri['infrastructure_index'] = df_negeri[infrastructure_cols].sum(axis=1)

# Handle potential NaN or infinite values in ratios for correlation calculation
df_negeri['teacher_gender_ratio'] = df_negeri['teacher_gender_ratio'].replace([np.inf, -np.inf], np.nan)
df_negeri['student_gender_ratio'] = df_negeri['student_gender_ratio'].replace([np.inf, -np.inf], float('nan'))
df_negeri['teacher_student_ratio'] = df_negeri['teacher_student_ratio'].replace([np.inf, -np.inf], float('nan'))


# 5. Korelasi awal antara sarpras, guru, dan akreditasi
print("\n=== Korelasi antara Infrastructure Index, PTK, PD, dan Ratios (Negeri Schools) ===")
correlation_cols = ['infrastructure_index', 'ptk', 'pd', 'teacher_student_ratio', 'student_gender_ratio', 'teacher_gender_ratio']
correlation_matrix = df_negeri[correlation_cols].corr()
display(correlation_matrix)


=== Korelasi antara Infrastructure Index, PTK, PD, dan Ratios (Negeri Schools) ===


,infrastructure_index,ptk,pd,teacher_student_ratio,student_gender_ratio,teacher_gender_ratio
infrastructure_index,1.000000,0.672335,0.715761,-0.081196,-0.057451,-0.071074
ptk,0.672335,1.000000,0.784553,-0.059486,-0.025312,-0.022164
pd,0.715761,0.784553,1.000000,-0.219018,-0.104071,-0.090358
teacher_student_ratio,-0.081196,-0.059486,-0.219018,1.000000,0.035826,0.167005
student_gender_ratio,-0.057451,-0.025312,-0.104071,0.035826,1.000000,0.008136
teacher_gender_ratio,-0.071074,-0.022164,-0.090358,0.167005,0.008136,1.000000


#Modeling & Analisis Lanjutan


## 1. Clustering (K-Means/Hierarchical)

In [ ]:
# Normalisasi teks kabupaten/kota & kecamatan
df['kabupaten/kota'] = df['kabupaten/kota'].str.lower().str.strip()
df['kecamatan'] = df['kecamatan'].str.lower().str.strip()

# Filter data for Negeri status only
df_negeri = df[df['status_sekolah'] == 'Negeri'].copy()

# Recalculate necessary columns after filtering
df_negeri['teacher_student_ratio'] = df_negeri['ptk'] / df_negeri['pd']
infrastructure_cols = [
    col for col in df_negeri.columns
    if 'ruang' in col or 'laboratorium' in col or 'perpustakaan' in col or 'bangunan' in col
]
df_negeri['infrastructure_index'] = df_negeri[infrastructure_cols].sum(axis=1)


# Select the features for clustering
features = ['teacher_student_ratio', 'infrastructure_index', 'pd']
X = df_negeri[features].copy()

# Handle potential NaN or infinite values in selected features before scaling and clustering
# For simplicity, we will fill NaN with the mean of the column.
# You might choose a different strategy based on your data.
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(X.mean(), inplace=True)


# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Determine the optimal number of clusters (e.g., using the Elbow method)
# For this example, let's assume we want 3 clusters as suggested by the output requirement
n_clusters = 3

# Apply K-Means clustering
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10) # Added n_init to suppress warning
kmeans.fit(X_scaled) # Train the model
df_negeri['cluster'] = kmeans.labels_

# Analyze the characteristics of each cluster
print("\n=== Cluster Analysis (Negeri Schools) ===")
cluster_summary = df_negeri.groupby('cluster')[features].mean()

# Handle potential inf in the cluster summary output
cluster_summary = cluster_summary.replace([np.inf, -np.inf], float('nan'))

display(cluster_summary)

# Map cluster labels to meaningful names (this requires domain knowledge and interpretation of the cluster_summary)
# For example, based on the means in cluster_summary:
# Cluster 0 might be "Sedang"
# Cluster 1 might be "Unggul"
# Cluster 2 might be "Butuh Perhatian"
# This mapping is illustrative and should be adjusted based on the actual cluster means.
cluster_names = {
    0: 'Unggul',
    1: 'Sedang',
    2: 'Butuh Perhatian'
}

df_negeri['cluster_name'] = df_negeri['cluster'].map(cluster_names)

print("\n=== Schools with Cluster Names (Sample - Negeri Schools) ===")
display(df_negeri[['sekolah', 'cluster', 'cluster_name'] + features].head())

# You can further analyze the distribution of other variables within each cluster
# For example, the distribution of 'status_sekolah' or 'akreditasi' within each cluster.


=== Cluster Analysis (Negeri Schools) ===


,teacher_student_ratio,infrastructure_index,pd
cluster,,,
0,0.191661,34.863014,65.041096
1,0.179295,81.380952,159.428571
2,4.000000,26.000000,1.000000



=== Schools with Cluster Names (Sample - Negeri Schools) ===


,sekolah,cluster,cluster_name,teacher_student_ratio,infrastructure_index,pd
2,SLB NEGERI KELEYAN BANGKALAN,0,Unggul,0.380000,34,50
11,SDLB ABCD PGRI 3 CLURING,2,Butuh Perhatian,4.000000,26,1
12,SLB PGRI 3 CLURING,0,Unggul,0.056818,38,88
13,SLB NEGERI TAMANAGUNG BANYUWANGI,1,Sedang,0.300971,70,103
14,SMALBS BHAKTI PERTIWI,0,Unggul,0.384615,26,13


##2.	Classification (Decision Tree/Random Forest/LogReg)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd

# Prepare the data for classification

# Filter data for Negeri status only
df_negeri = df[df['status_sekolah'] == 'Negeri'].copy()

# Recalculate necessary columns after filtering
df_negeri['teacher_student_ratio'] = df_negeri['ptk'] / df_negeri['pd']
infrastructure_cols = [
    col for col in df_negeri.columns
    if 'ruang' in col or 'laboratorium' in col or 'perpustakaan' in col or 'bangunan' in col
]
df_negeri['infrastructure_index'] = df_negeri[infrastructure_cols].sum(axis=1)


# Drop rows where 'akreditasi' is missing, as it's our target variable
df_classified = df_negeri.dropna(subset=['akreditasi']).copy()

# Identify and remove classes with only one member for stratified splitting
class_counts = df_classified['akreditasi'].value_counts()
rare_classes = class_counts[class_counts < 2].index
df_classified = df_classified[~df_classified['akreditasi'].isin(rare_classes)].copy()


# Define features (X) and target (y)
# Re-including 'status_sekolah' to be handled as a categorical feature
features = ['ptk', 'teacher_student_ratio', 'infrastructure_index'] # 'jumlah guru' is 'ptk'
target = 'akreditasi'

X = df_classified[features].copy() # Create a copy to avoid the warning
y = df_classified[target].copy() # Create a copy to avoid the warning

# Handle potential NaN or infinite values in selected features
X.replace([np.inf, -np.inf], np.nan, inplace=True)
# For simplicity, fill missing numerical values with the mean
numerical_features = ['ptk', 'teacher_student_ratio', 'infrastructure_index']
X.loc[:, numerical_features] = X.loc[:, numerical_features].fillna(X.loc[:, numerical_features].mean()) # Use .loc for assignment


# Encode target (akreditasi)
# Use LabelEncoder for the target
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Define preprocessing for numerical and categorical features
# Since we filtered for Negeri schools, 'status_sekolah' is not needed as a feature.
# The original features defined in 'features' are all numerical.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features) # No transformation needed for numerical features
    ])


# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded) # Added stratify=y_encoded


# Create and train classification models

# Decision Tree
dt_model = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', DecisionTreeClassifier(random_state=42))])
dt_model.fit(X_train, y_train)
dt_pred = dt_model.predict(X_test)

print("=== Laporan Klasifikasi Decision Tree (Negeri Schools) ===")
print(classification_report(y_test, dt_pred, target_names=label_encoder.classes_))

# Random Forest
rf_model = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier(random_state=42))])
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

print("\n=== Laporan Klasifikasi Random Forest (Negeri Schools) ===")
print(classification_report(y_test, rf_pred, target_names=label_encoder.classes_))

# Logistic Regression (Note: Might not be suitable for multi-class classification with non-linear relationships)
# lr_model = Pipeline(steps=[('preprocessor', preprocessor),
#                            ('classifier', LogisticRegression(random_state=42, max_iter=1000))])
# lr_model.fit(X_train, y_train)
# lr_pred = lr_model.predict(X_test)

# print("\n=== Laporan Klasifikasi Regresi Logistik (Negeri Schools) ===")
# print(classification_report(y_test, lr_pred, target_names=label_encoder.classes_))


# Analyze feature importance (for Decision Tree and Random Forest)
print("\n=== Kepentingan Fitur (Random Forest - Negeri Schools) ===")
# Get feature names from the preprocessor
# In this case, since only numerical features are passed through, the feature names are the same as the original numerical features
all_feature_names = numerical_features

# Access the trained Random Forest classifier from the pipeline
rf_classifier = rf_model.named_steps['classifier']
feature_importances = pd.Series(rf_classifier.feature_importances_, index=all_feature_names)
display(feature_importances.sort_values(ascending=False))

=== Laporan Klasifikasi Decision Tree (Negeri Schools) ===
              precision    recall  f1-score   support

           A       0.75      0.86      0.80         7
           B       0.80      0.80      0.80        10
           C       0.00      0.00      0.00         1

    accuracy                           0.78        18
   macro avg       0.52      0.55      0.53        18
weighted avg       0.74      0.78      0.76        18


=== Laporan Klasifikasi Random Forest (Negeri Schools) ===
              precision    recall  f1-score   support

           A       0.78      1.00      0.88         7
           B       0.89      0.80      0.84        10
           C       0.00      0.00      0.00         1

    accuracy                           0.83        18
   macro avg       0.56      0.60      0.57        18
weighted avg       0.80      0.83      0.81        18


=== Kepentingan Fitur (Random Forest - Negeri Schools) ===


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/m

,0
ptk,0.343585
infrastructure_index,0.336033
teacher_student_ratio,0.320382


##3.	Regression (Linear/Multiple Regression)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Prepare the data for regression

# Filter data for Negeri status only
df_negeri = df[df['status_sekolah'] == 'Negeri'].copy()

# Recalculate necessary columns after filtering
infrastructure_cols = [
    col for col in df_negeri.columns
    if 'ruang' in col or 'laboratorium' in col or 'perpustakaan' in col or 'bangunan' in col
]
df_negeri['infrastructure_index'] = df_negeri[infrastructure_cols].sum(axis=1)


# Define dependent variable (target) and independent variables (features)
target = 'pd' # jumlah siswa
features = ['ptk', 'infrastructure_index'] # jumlah guru, sarpras, removed status sekolah as it's filtered

# Drop rows where the target variable is missing (although based on previous checks, 'pd' has no missing values)
df_reg = df_negeri.dropna(subset=[target]).copy()

# Select features and target
X = df_reg[features].copy()
y = df_reg[target].copy()

# Handle potential NaN or infinite values in selected features
X.replace([np.inf, -np.inf], np.nan, inplace=True)
# For simplicity, fill missing numerical values with the mean
numerical_features = ['ptk', 'infrastructure_index']
X.loc[:, numerical_features] = X.loc[:, numerical_features].fillna(X.loc[:, numerical_features].mean())

# No categorical features to encode after filtering


# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Linear Regression model
model = LinearRegression() # Removed preprocessor
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\n=== Evaluasi Model Regresi Linear (Negeri Schools) ===")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R2): {r2:.2f}")

# Analyze the coefficients to understand the influence of each feature
print("\n=== Koefisien Model Regresi (Negeri Schools) ===")
# Get feature names
all_feature_names = features

coefficients = pd.Series(model.coef_, index=all_feature_names)
display(coefficients.sort_values(ascending=False))

# Interpretation of coefficients:
# For numerical features (ptk, infrastructure_index), the coefficient represents the change in 'pd' for a one-unit increase in the feature, holding other features constant.
# For one-hot encoded categorical features (status_sekolah), the coefficient represents the difference in 'pd' compared to the reference category (which is implicitly handled by OneHotEncoder's drop='first' or the absence of one category).


=== Evaluasi Model Regresi Linear (Negeri Schools) ===
Mean Squared Error (MSE): 1016.26
Root Mean Squared Error (RMSE): 31.88
R-squared (R2): 0.65

=== Koefisien Model Regresi (Negeri Schools) ===


,0
ptk,2.911066
infrastructure_index,0.702419


#Interpretasi & Insight

blalba

#Visualisasi & Storytelling

ballalba

#Deliverables untuk Lomba

dadsda

In [ ]:
slb_district_counts = df['kabupaten/kota'].value_counts().reset_index()
slb_district_counts.columns = ['kabupaten/kota', 'jumlah_slb']
display(slb_district_counts.head())

**Reasoning**:
Calculate the mean infrastructure index per accreditation level for the heatmap visualization, handling missing values.



In [ ]:
# Filter data for Negeri status only
df_negeri = df[df['status_sekolah'] == 'Negeri'].copy()

# Recalculate infrastructure index for the filtered dataframe
infrastructure_cols = [
    col for col in df_negeri.columns
    if 'ruang' in col or 'laboratorium' in col or 'perpustakaan' in col or 'bangunan' in col
]
df_negeri['infrastructure_index'] = df_negeri[infrastructure_cols].sum(axis=1)

# Handle missing 'akreditasi' values by dropping rows
df_negeri_cleaned = df_negeri.dropna(subset=['akreditasi']).copy()

# Group by 'akreditasi' and calculate the mean 'infrastructure_index'
accreditation_infrastructure_agg = df_negeri_cleaned.groupby('akreditasi')['infrastructure_index'].mean().reset_index()
display(accreditation_infrastructure_agg.head())

**Reasoning**:
Calculate the mean teacher-student ratio per district for the bar chart visualization, handling infinite and NaN values.



In [ ]:
# Filter data for Negeri status only
df_negeri = df[df['status_sekolah'] == 'Negeri'].copy()

# Recalculate teacher student ratio for the filtered dataframe
df_negeri['teacher_student_ratio'] = df_negeri['ptk'] / df_negeri['pd']

# Handle potential NaN or infinite values in 'teacher_student_ratio'
# Replace infinite values with NaN
df_negeri['teacher_student_ratio'] = df_negeri['teacher_student_ratio'].replace([np.inf, -np.inf], np.nan)
# Fill NaN values with the mean of the column
df_negeri['teacher_student_ratio'].fillna(df_negeri['teacher_student_ratio'].mean(), inplace=True)


# Group by 'kabupaten/kota' and calculate the mean 'teacher_student_ratio'
teacher_student_ratio_district = df_negeri.groupby('kabupaten/kota')['teacher_student_ratio'].mean().reset_index()
display(teacher_student_ratio_district.head())

## Generate visual


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Filter data for Negeri status only
df_negeri = df[df['status_sekolah'] == 'Negeri'].copy()

# Recalculate infrastructure index for the filtered dataframe
infrastructure_cols = [
    col for col in df_negeri.columns
    if 'ruang' in col or 'laboratorium' in col or 'perpustakaan' in col or 'bangunan' in col
]
df_negeri['infrastructure_index'] = df_negeri[infrastructure_cols].sum(axis=1)

# Handle missing 'akreditasi' values by dropping rows
df_negeri_cleaned = df_negeri.dropna(subset=['akreditasi']).copy()

# Group by 'akreditasi' and calculate the mean 'infrastructure_index'
accreditation_infrastructure_agg = df_negeri_cleaned.groupby('akreditasi')['infrastructure_index'].mean().reset_index()


# 2. Heatmap of accreditation vs. infrastructure
print("\n=== Heatmap of Accreditation vs. Infrastructure Index (Negeri Schools) ===")

# Sort the accreditation levels for better visualization order (optional, adjust as needed)
accreditation_order = ['Belum Akreditasi', 'C', 'B', 'A']
accreditation_infrastructure_agg['akreditasi'] = pd.Categorical(accreditation_infrastructure_agg['akreditasi'], categories=accreditation_order, ordered=True)
accreditation_infrastructure_agg = accreditation_infrastructure_agg.sort_values('akreditasi')


plt.figure(figsize=(8, 6))
sns.heatmap(accreditation_infrastructure_agg.set_index('akreditasi'), annot=True, fmt=".2f", cmap="YlGnBu", cbar_kws={'label': 'Average Infrastructure Index'})
plt.title('Average Infrastructure Index by Accreditation Level (Negeri Schools)')
plt.xlabel('Feature')
plt.ylabel('Accreditation Level')
plt.yticks(rotation=0) # Ensure y-axis labels are horizontal
plt.show()

In [ ]:
# 3. Bar chart of teacher-student ratio per district
print("\n=== Average Teacher-Student Ratio per District (Negeri Schools) ===")

# Sort the data by teacher_student_ratio for better readability
teacher_student_ratio_district_sorted = teacher_student_ratio_district.sort_values('teacher_student_ratio', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='kabupaten/kota', y='teacher_student_ratio', data=teacher_student_ratio_district_sorted, palette='viridis')
plt.title('Average Teacher-Student Ratio per District (Negeri Schools)')
plt.xlabel('District/City')
plt.ylabel('Average Teacher-Student Ratio')
plt.xticks(rotation=90) # Rotate x-axis labels for better readability
plt.tight_layout() # Adjust layout to prevent labels overlapping
plt.show()

In [ ]:
# 4. Pie chart of student gender distribution
print("\n=== Student Gender Distribution (All Schools) ===")

# Filter data for Negeri status only
df_negeri = df[df['status_sekolah'] == 'Negeri'].copy()

# Calculate total male and female students for Negeri schools
total_male_students = df_negeri['pd_laki'].sum()
total_female_students = df_negeri['pd_perempuan'].sum()

labels = ['Male Students', 'Female Students']
sizes = [total_male_students, total_female_students]
colors = ['#ff9999','#66b3ff']
explode = (0.1, 0)  # explode 1st slice (Male Students)

plt.figure(figsize=(8, 8))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
        shadow=True, startangle=140)
plt.title('Student Gender Distribution (Negeri Schools)') # Updated title
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

**Reasoning**:
Visualize the trained Decision Tree model to understand its structure and how it makes predictions.



In [ ]:
from sklearn.tree import export_graphviz
import graphviz
from sklearn.pipeline import Pipeline # Import Pipeline

# 5. Decision Tree visualization
print("\n=== Decision Tree Visualization (Negeri Schools) ===")

# Access the trained Decision Tree classifier directly (dt_model is not a Pipeline)
# IMPORTANT: The dt_model used here was trained in a previous step (cell tg3d_jimbDrM)
# on data that was already filtered to include only 'Negeri' status schools (df_negeri).
# Therefore, this visualization is inherently based on Negeri schools data.
dt_classifier = dt_model # Access the classifier directly


# Get the feature names from the preprocessor that was used before training the model
# In the cell where the Decision Tree model was trained (tg3d_jimbDrM), the numerical features
# were passed through without transformation, so we can use the original feature names.
feature_names = ['ptk', 'teacher_student_ratio', 'infrastructure_index']


# Export the decision tree to a DOT file
dot_data = export_graphviz(dt_classifier, out_file=None,
                           feature_names=feature_names, # Use the correct feature names
                           class_names=label_encoder.classes_, # Use label_encoder.classes_ for target names
                           filled=True, rounded=True,
                           special_characters=True)

# Render the DOT data using Graphviz
graph = graphviz.Source(dot_data)

# Display the graph (this will render it as an image in the notebook)
display(graph)

**Reasoning**:
Recalculate the mean teacher-student ratio per district for the bar chart visualization, handling infinite and NaN values, and ensure it's filtered for Negeri schools.